# 기본 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 32.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (se

In [ ]:
#전처리 패키지
import pandas as pd
import numpy as np
from datetime import datetime
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import spacy
import string

#토픽 모델링 패키지
from bertopic import BERTopic
import tensorflow_hub
from umap import UMAP
import hdbscan
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 데이터 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NH투자증권 공모전/data/df_0915.csv', encoding='latin1')
df.head(5)

,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo,all_tck_iem_cd
0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...,['NGS']
1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...,['PFX']
2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,"['TYG', 'FEN']"
3,20230102,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...,['NVIV']
4,20230102,NUVL,Are Medical Stocks Lagging DICE Therapeutics ...,"Stocks,Investing",Zacks,Investors interested in Medical stocks should ...,"DICE,NUVL",https://www.nasdaq.com/articles/are-medical-st...,"['NUVL', 'DICE']"


In [ ]:
df.shape

(88769, 9)

In [ ]:
#1월 데이터만 추출
df['year_month'] = df['rgs_dt'] // 100
Jan_df = df[df['year_month'] == 202301]
Jan_df

,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo,all_tck_iem_cd,year_month
0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...,['NGS'],202301
1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...,['PFX'],202301
2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,"['TYG', 'FEN']",202301
3,20230102,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...,['NVIV'],202301
4,20230102,NUVL,Are Medical Stocks Lagging DICE Therapeutics ...,"Stocks,Investing",Zacks,Investors interested in Medical stocks should ...,"DICE,NUVL",https://www.nasdaq.com/articles/are-medical-st...,"['NUVL', 'DICE']",202301
...,...,...,...,...,...,...,...,...,...,...
1526,20230131,LUNG,BlackRock Cuts Stake in ProLung (LUNG),Stocks,Fintel,Fintel reports that BlackRock has filed a 13G/...,LUNG,https://www.nasdaq.com/articles/blackrock-cuts...,['LUNG'],202301
1527,20230131,SMBK,SmartFinancial (SMBK) Declares $0.08 Dividend,Dividends,Fintel,"SmartFinancial said on January 26, 2023 that i...",SMBK,https://www.nasdaq.com/articles/smartfinancial...,['SMBK'],202301
1528,20230131,PGC,Peapack Gladstone Financial (PGC) Declares $0....,Dividends,Fintel,Peapack Gladstone Financial said on January 26...,PGC,https://www.nasdaq.com/articles/peapack-gladst...,['PGC'],202301
1529,20230131,HRTX,Franklin Resources Cuts Stake in Heron Therape...,Stocks,Fintel,Fintel reports that Franklin Resources has fil...,HRTX,https://www.nasdaq.com/articles/franklin-resou...,['HRTX'],202301


In [ ]:
#단순 주가 정보만 제공하는 뉴스 제거(Pre-Market|After-Hours)
Jan_df = Jan_df[~Jan_df['ctgy_cfc_ifo'].str.contains("Pre-Market|After-Hours")]
Jan_df

,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo,all_tck_iem_cd,year_month
0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...,['NGS'],202301
1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...,['PFX'],202301
2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,"['TYG', 'FEN']",202301
3,20230102,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...,['NVIV'],202301
4,20230102,NUVL,Are Medical Stocks Lagging DICE Therapeutics ...,"Stocks,Investing",Zacks,Investors interested in Medical stocks should ...,"DICE,NUVL",https://www.nasdaq.com/articles/are-medical-st...,"['NUVL', 'DICE']",202301
...,...,...,...,...,...,...,...,...,...,...
1526,20230131,LUNG,BlackRock Cuts Stake in ProLung (LUNG),Stocks,Fintel,Fintel reports that BlackRock has filed a 13G/...,LUNG,https://www.nasdaq.com/articles/blackrock-cuts...,['LUNG'],202301
1527,20230131,SMBK,SmartFinancial (SMBK) Declares $0.08 Dividend,Dividends,Fintel,"SmartFinancial said on January 26, 2023 that i...",SMBK,https://www.nasdaq.com/articles/smartfinancial...,['SMBK'],202301
1528,20230131,PGC,Peapack Gladstone Financial (PGC) Declares $0....,Dividends,Fintel,Peapack Gladstone Financial said on January 26...,PGC,https://www.nasdaq.com/articles/peapack-gladst...,['PGC'],202301
1529,20230131,HRTX,Franklin Resources Cuts Stake in Heron Therape...,Stocks,Fintel,Fintel reports that Franklin Resources has fil...,HRTX,https://www.nasdaq.com/articles/franklin-resou...,['HRTX'],202301


# 뉴스 기사 추출

In [ ]:
!pip install newspaper3k

In [ ]:
#뉴스 기사 여러개 가져오는 함수
from newspaper import Article
def get_articles(urls):
    articles = []

    for url in urls:
        article = Article(url)
        article.download()
        article.parse()
        text = article.text
        articles.append(text)

    return articles

In [ ]:
# 뉴스 기사 가져오기
Jan_urls = Jan_df['url_ifo']
Jan_articles = get_articles(Jan_urls)

In [ ]:
# 정규 표현식을 사용하여 마지막 문장 제외
Jan_articles_filter = []

for article in Jan_articles:
    filtered_article = re.sub(r'\.[^.]*$', '', article)
    Jan_articles_filter.append(filtered_article)

# 불용어 제거

In [ ]:
#형태소 분석 모델 다운로드
!python -m spacy download en_core_web_lg

In [ ]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_lg")#trf 모델로 변경가능
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

In [ ]:
#불용어 리스트
print(spacy_stopwords_list)

['ours', 'you', 'as', 'was', 'still', 'themselves', 'to', 'are', 'why', 'amongst', 'has', 'wherein', 'without', 'between', 'myself', 'on', 'does', 'via', '’s', 'make', 'twenty', 'fifteen', '’ll', 'give', 'out', 'or', 'not', 'become', 'whenever', 'their', "n't", 'after', 'whereupon', 'almost', '’re', 'anyway', 'due', 'seems', 'seemed', 'while', 'against', 'moreover', 'most', 'of', 'therein', 'this', 'itself', 'those', 'rather', 'now', 'third', '‘ll', 'doing', 'part', 'yet', 'namely', 'quite', 'although', 'fifty', 'see', 'such', 'can', 'others', 'hereupon', 'latter', 'anything', 'put', 'n’t', 'all', 'before', '‘m', 'again', 'off', 'beyond', 'for', 'under', 'else', 'side', 'least', '‘re', 'everything', 'becomes', 'formerly', 'several', 'during', 'upon', 'which', 'last', 'these', 'each', 'do', 'very', 'used', 'thereafter', 'where', 'otherwise', 'former', 'yourselves', 'amount', 'below', 'elsewhere', 'into', 'her', 'towards', 'serious', 'same', 'nowhere', 'someone', 'noone', 'around', 'up',

In [ ]:
umap_model = UMAP(n_neighbors=15, min_dist=0.1, n_components=2,random_state=42,metric='cosine') #umap_model 정의
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean',cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=embedding_model,top_n_words=10,min_topic_size=10,umap_model=umap_model,hdbscan_model=hdbscan_model)#,seed_topic_list=seed_topic_list)
#데이터 좀 많을 떄 min_topic_size=10 , min_cluster_Size=6 등으로 조절

In [ ]:
topics, probabillities= topic_model.fit_transform(Jan_articles_filter)

  0%|          | 0/1531 [00:00<?, ?it/s]WARNING:tensorflow:From /usr/local/lib/python3.10/dist-packages/bertopic/backend/_use.py:54: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity with explicit device placement instead.
100%|██████████| 1531/1531 [00:14<00:00, 102.78it/s]
2023-09-15 16:22:21,623 - BERTopic - Transformed documents to Embeddings
2023-09-15 16:22:38,547 - BERTopic - Reduced dimensionality
2023-09-15 16:22:38,631 - BERTopic - Clustered reduced embeddings


In [ ]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON은 제거
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE','PERSON']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words

In [ ]:
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(Jan_articles_filter, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [ ]:
topic_model.get_topic_info() #전체 정보

In [ ]:
topic_model.get_topic(0)# 토픽 하나씩 뽑아서 키워드 체크

[('spdr fund', 0.037751949237413096),
 ('select sector', 0.03443798892287216),
 ('sector spdr', 0.034256212620034455),
 ('spdr', 0.03083317246107826),
 ('say', 0.02624988070503557),
 ('select', 0.025907291523259678),
 ('nasdaq technology', 0.017614419584454265),
 ('consumer', 0.016947416596085715),
 ('company news', 0.016848250739838533),
 ('iq', 0.016607905095659146)]

In [ ]:
topic_model.get_topic(1)

[('health care', 0.05752266473897738),
 ('care', 0.03903499996350809),
 ('health', 0.03874243060001576),
 ('drug', 0.03263763442343928),
 ('care stock', 0.030273356634876014),
 ('spdr fund', 0.022189414485113662),
 ('say', 0.02146148306888555),
 ('treatment', 0.02143626825867632),
 ('trial', 0.02072220956626332),
 ('patient', 0.020549602625838662)]

In [ ]:
similar_topics, similarity = topic_model.find_topics("bitcoin", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기

[37, 13, 32, 68, 49]

In [ ]:
topic_model.get_topic(37)

[('crypto', 0.11096873725272503),
 ('ethereum', 0.07139287114476711),
 ('token', 0.061377078269901963),
 ('blockchain', 0.0584584265759942),
 ('fool', 0.035058094224645696),
 ('stablecoin', 0.033486678439835785),
 ('tron', 0.030439697524721802),
 ('cryptocurrencie', 0.029403175347351287),
 ('stock advisor', 0.027410589433742187),
 ('cryptocurrency', 0.026646732357096876)]

In [ ]:
topic_model.get_topic(32)

[('hammer', 0.0495157302940914),
 ('rsi', 0.03965524902216175),
 ('trend reversal', 0.03748663041516425),
 ('pattern', 0.03712137998095849),
 ('reversal', 0.03551658107287668),
 ('selling pressure', 0.028518833978918797),
 ('oscillate', 0.028470087252722192),
 ('turnaround', 0.027618304900199966),
 ('downtrend', 0.025658485284293243),
 ('oversell', 0.024405528065799556)]

In [ ]:
similar_topics, similarity = topic_model.find_topics("battery", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기

[17, 15, 50, 22, 32]

In [ ]:
topic_model.get_topic(17)

[('barrel', 0.03822849560629332),
 ('coal', 0.03510367304053004),
 ('sector index', 0.03296441593291943),
 ('energy', 0.031652849255700034),
 ('say', 0.030947658340779612),
 ('hydrogen', 0.027414239495871046),
 ('xle', 0.022103016002922683),
 ('energy select', 0.022103016002922683),
 ('oil', 0.02205391158089298),
 ('oil service', 0.020356523193796845)]

In [ ]:
topic_model.get_topic(15)

[('packaging', 0.022156934431953153),
 ('industry', 0.01920862761824916),
 ('paper', 0.016371692925043563),
 ('zacks', 0.014197142724548488),
 ('satellite', 0.012681902937188048),
 ('industry participant', 0.012281259464560369),
 ('demand', 0.01099439929162409),
 ('product', 0.010638657464872357),
 ('industry trade', 0.009873390246006569),
 ('service', 0.00913986713481902)]

In [ ]:
similar_topics, similarity = topic_model.find_topics("TSLA", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기

[62, 20, 9, 6, 12]

In [ ]:
topic_model.get_topic(62)

[('dividend', 0.056036625061693296),
 ('retirement', 0.055071928594323305),
 ('yield', 0.045595287213324925),
 ('bond', 0.04318121805464133),
 ('retiree', 0.039367566454366676),
 ('yield s&p', 0.0363460466811399),
 ('s&p yield', 0.03380517299693314),
 ('dividend growth', 0.03204928442810833),
 ('annualize dividend', 0.029324162078255164),
 ('nrg', 0.028708455380735513)]

In [ ]:
topic_model.get_representative_docs(1)[2] # 토픽에 속한 문서 내용 최대 3개까지 볼 수 있음

'Health care stocks were narrowly lower Thursday afternoon, with the NYSE Health Care Index slipping 0.3% while the Health Care Select Sector SPDR Fund (XLV) was down just 0.l%.\n\nThe iShares Biotechnology ETF (IBB), however, was climbing 0.4%.\n\nIn company news, Oramed (ORMP) plunged as much as 78%, sinking to a record low of $2.32 a share, after saying it was ending work on an oral insulin therapy for patients with type 2 diabetes after its ORMD-0801 drug candidate failed to meet its primary and secondary endpoints during a phase 3 trial.\n\nAkili (AKLI) fell 6% after the digital medicine company said it is eliminating 46 jobs - or about 30% of its current work force - in a bid to lower its operating costs. Akili also is expecting between $1.5 million to $2.5 million in severance and other termination-related costs in Q1.\n\nSimulations Plus (SLP) shares rose 1.4% after the drug discovery firm announced an accelerated stock buyback agreement to reacquire $20 million shares from sel

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

# 불용어 제거 2(미디어, 돈, 국가 제외)

In [ ]:
media = list(df['mdi_ifo'].unique())
media_lower = [m.lower() for m in media]

In [ ]:
#추가 불용어리스트
stop_words_list=[media, media_lower, "spdr", "spdr fund", "etf", "investorplace", "click", "zacks"]
for word in stop_words_list:
    spacy_stopwords_list.append(word)

In [ ]:
#처음 토픽을 찾을 때 활용하긴 적합하지 않고 토픽을 뽑아낸 후 한번더 거르고 싶을 때 사용하는 것이 좋음
seed_topic_list=[['agricultural'], ['buliding'],['wood','paper'],['advertising'],
               ['chemicals'], ['metals','mining'],['agencies'],['gaming'],['multimeida'],
               ['entertainment'],['telecommunications'],['automotive'],['engineering'],['construction'],['consumer','goods'],['food','beverage']
               ,['leisure','travel'],['shipping'],['transportation'],['personal','personal services'],['retail'],['education'],['training'],
               ['energy'],['asset management','asset'],['banks'],['capital markets','capital'],['financial services','financial'],['insurance'],['mortgage finance','mortgage'],['biotechnology'],
               ['healthcare services','healthcare'],['medical devices','medical'],['pharmaceutical'],['aerospace'],['defense'],['airports','airlines'],['industrials'],['equipment','machinery'],
               ['professional','business services','business'],['rental','leasing services','leasing'],['real estate'],['electronics','computers'],['technology'],['semiconductors'],['software'],['utilities']]

In [ ]:
topic_model = BERTopic(verbose=True, embedding_model=embedding_model,top_n_words=10,min_topic_size=10,umap_model=umap_model,hdbscan_model=hdbscan_model, seed_topic_list=seed_topic_list)

In [ ]:
topics, probabillities= topic_model.fit_transform(Jan_articles_filter)

In [ ]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON, 'MONEY','GPE'은 제거
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE','PERSON','MONEY','GPE']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words

In [ ]:
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(Jan_articles_filter, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [ ]:
topic_model.get_topic_info() #전체 정보

,Topic,Count,Name,Representation,Representative_Docs
0,-1,179,-1_oz_au_ft_ag,"[oz, au, ft, ag, lithium, company, oz ag, divi...",[Eagle Bancorp (EGBN) came out with quarterly ...
1,0,85,0_share represent_ownership_filing firm_portfo...,"[share represent, ownership, filing firm, port...",[Fintel reports that BlackRock Inc. has filed ...
2,1,84,1_health care_care_health_drug,"[health care, care, health, drug, care stock, ...",[Health care stocks were moderately higher thi...
3,2,69,2_earning_zacks_estimate_revision,"[earning, zacks, estimate, revision, earning s...",[Capital Bancorp (CBNK) came out with quarterl...
4,3,47,3_earning_zacks_estimate_revision,"[earning, zacks, estimate, revision, consensus...",[Summit Financial (SMMF) came out with quarter...
...,...,...,...,...,...
56,55,7,55_price target_target_deviation_price,"[price target, target, deviation, price, analy...",[Insmed (INSM) closed the last trading session...
57,56,6,56_earning_zacks_estimate_cnb,"[earning, zacks, estimate, cnb, earning share,...",[CNB Financial (CCNE) came out with quarterly ...
58,57,5,57_asx_miner_gold_santo,"[asx, miner, gold, santo, wds.ax santo, wds.ax...",[Jan 19 (Reuters) - Australian shares slipped ...
59,58,5,58_semiconductor index_technology select_xlk_f...,"[semiconductor index, technology select, xlk, ...","[Technology stocks were advancing on Friday, r..."


In [ ]:
topic_model.get_topic(0)# 토픽 하나씩 뽑아서 키워드 체크

[('share represent', 0.05900068386926435),
 ('ownership', 0.046106837654032846),
 ('filing firm', 0.045586178659345285),
 ('portfolio allocation', 0.04553497915448075),
 ('report share', 0.045520381467744786),
 ('ownership company', 0.04431660541450322),
 ('represent ownership', 0.04419214973898987),
 ('company filing', 0.042883199005866214),
 ('firm report', 0.04135674324472821),
 ('represent', 0.03992104927772406)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

# 불용어 사전 바꾸기(임베딩모델 = spaCy, hdbscan metric = cosine)

In [ ]:
!pip install spacy-transformers

In [ ]:
#형태소 분석 모델 다운로드
!python -m spacy download en_core_web_trf

2023-09-16 03:16:54.427686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.2
    Uninstalling transformers-4.33.2:
      Successfully uninstalled transformers-4.33.2
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
!python -m spacy download en_core_web_lg

2023-09-16 03:19:56.616683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_lg")
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

In [ ]:
#추가 불용어리스트
media = list(df['mdi_ifo'].unique())
media_lower = [m.lower() for m in media]
stop_words_list=[media_lower, "spdr", "spdr fund", "etf", "investorplace", "click", "zacks"]
for word in stop_words_list:
    spacy_stopwords_list.append(word)

In [ ]:
seed_topic_list=[['agricultural'], ['buliding'],['wood','paper'],['advertising'],
               ['chemicals'], ['metals','mining'],['agencies'],['gaming'],['multimeida'],
               ['entertainment'],['telecommunications'],['automotive'],['engineering'],['construction'],['consumer','goods'],['food','beverage']
               ,['leisure','travel'],['shipping'],['transportation'],['personal','personal services'],['retail'],['education'],['training'],
               ['energy'],['asset management','asset'],['banks'],['capital markets','capital'],['financial services','financial'],['insurance'],['mortgage finance','mortgage'],['biotechnology'],
               ['healthcare services','healthcare'],['medical devices','medical'],['pharmaceutical'],['aerospace'],['defense'],['airports','airlines'],['industrials'],['equipment','machinery'],
               ['professional','business services','business'],['rental','leasing services','leasing'],['real estate'],['electronics','computers'],['technology'],['semiconductors'],['software'],['utilities']]
## 나스닥 기업의 섹터 여기다 설정하기

In [ ]:
umap_model = UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42,metric='cosine') #umap_model 정의
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=5, metric='cosine', cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

#embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=nlp, top_n_words=10,min_topic_size=10,umap_model=umap_model,hdbscan_model=hdbscan_model, seed_topic_list=seed_topic_list)
#데이터 좀 많을 떄 min_topic_size=10 , min_cluster_Size=6 등으로 조절

In [ ]:
topics, probabillities= topic_model.fit_transform(Jan_articles_filter)

100%|██████████| 1531/1531 [04:24<00:00,  5.80it/s]
2023-09-16 03:31:28,722 - BERTopic - Transformed documents to Embeddings
  2%|▏         | 1/47 [00:00<00:00, 55.77it/s]


AttributeError: ignored

In [ ]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON은 제거
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in spacy_stopwords_list and token.ent_type_ not in ['TIME','CARDINAL','DATE','PERSON']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words